**Project Goal:**  
Clustering employees based on their wellbeing in connection to work conditions 

**Project Description:**  
Using the dataset from Kaglle: https://www.kaggle.com/datasets/waqi786/remote-work-and-mental-health


In [1]:
#Importing the libraries
import pandas as pd

In [7]:
#Importing the dataset
df = pd.read_csv("Impact_of_Remote_Work_on_Mental_Health.csv")

**Step 1** Exploratory Data Analysis


In [8]:
print(df.head())
print(df.info())

  Employee_ID  Age      Gender           Job_Role    Industry  \
0     EMP0001   32  Non-binary                 HR  Healthcare   
1     EMP0002   40      Female     Data Scientist          IT   
2     EMP0003   59  Non-binary  Software Engineer   Education   
3     EMP0004   27        Male  Software Engineer     Finance   
4     EMP0005   49        Male              Sales  Consulting   

   Years_of_Experience Work_Location  Hours_Worked_Per_Week  \
0                   13        Hybrid                     47   
1                    3        Remote                     52   
2                   22        Hybrid                     46   
3                   20        Onsite                     32   
4                   32        Onsite                     35   

   Number_of_Virtual_Meetings  Work_Life_Balance_Rating Stress_Level  \
0                           7                         2       Medium   
1                           4                         1       Medium   
2             

In [9]:
print(df.describe(include='all'))

       Employee_ID          Age  Gender         Job_Role Industry  \
count         5000  5000.000000    5000             5000     5000   
unique        5000          NaN       4                7        7   
top        EMP5000          NaN  Female  Project Manager  Finance   
freq             1          NaN    1274              738      747   
mean           NaN    40.995000     NaN              NaN      NaN   
std            NaN    11.296021     NaN              NaN      NaN   
min            NaN    22.000000     NaN              NaN      NaN   
25%            NaN    31.000000     NaN              NaN      NaN   
50%            NaN    41.000000     NaN              NaN      NaN   
75%            NaN    51.000000     NaN              NaN      NaN   
max            NaN    60.000000     NaN              NaN      NaN   

        Years_of_Experience Work_Location  Hours_Worked_Per_Week  \
count           5000.000000          5000            5000.000000   
unique                  NaN        